## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en analysant la rentabilité brute dans les **principales villes étudiantes françaises** (studios et T1 ≤45m²)

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🔍 Besoins data de Léa
- Évaluer le **taux de vacance locative** en France pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Visualiser les **villes à forte concentration étudiante** en France
- Analyser l'**évolution du prix au m² à l'achat et des loyers étudiants** en France
- Etudier **la rentabilité moyenne en France** en 2024
- Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux)
- Comparer **les quartiers les plus rentables (rentabilité brute)** à ?
- Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?
- Analyser la **localisation des universités/grandes écoles** à ?
- Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? 
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Analyser la **rentabilité locative brute** pour appartements étudiants ≤45m² dans **23 grandes villes françaises**
2. Explorer visuellement les **villes à forte concentration étudiante** et analyser les **taux de vacance locative**
3. Obtenir un **classement des villes** par rentabilité, prix et loyers pour décider où investir avec 200k€

## 📚 Import des bibliothèques ##

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests, zipfile
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.express as px
import seaborn as sns
from unidecode import unidecode
import glob

In [6]:
# ============================================================================
# SECTION : ANALYSE DE RENTABILITÉ LOCATIVE
# ============================================================================

# Import des fonctions depuis le module
from rentabilite_functions import *

print("Fonctions d'analyse de rentabilité chargées")

# Optionnel : Vous pouvez quand même exécuter l'analyse ici si besoin
# resultats = analyse_complete_rentabilite()

Fonctions d'analyse de rentabilité chargées


### Visualiser les zones à forte concentration étudiante en France ###

In [ ]:
url_df_pop_communales = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx"

df_populations_communales = pd.read_excel(url_df_pop_communales)

df_populations_communales.head()

In [ ]:
df_populations_communales.drop(['p13_pop', 'p14_pop', 'p15_pop', 'p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop'], axis=1, inplace=True)

# Garde les lignes où la colonne 'dep' contient le mot "Arrondissement"
df_filtered = df_populations_communales[df_populations_communales['libgeo'].str.contains("Arrondissement")]

df_filtered

In [ ]:
df = df_populations_communales.copy()

df['libgeo'] = df['libgeo'].replace(
    to_replace = [r'Paris.*', r'Lyon.*', r'Marseille.*'], 
    value = ['Paris', 'Lyon', 'Marseille'], 
    regex=True
)

treated_df = df.groupby('libgeo', as_index=False).agg({
    'objectid': 'first',   # garde le premier ID (ou autre logique)
    'reg': 'first',        # garde la première région
    'dep': 'first',        # idem pour département
    'cv': 'first', 
    'codgeo': 'first',
    'p21_pop': 'sum'
})

treated_df

In [ ]:
df_pop_100k = treated_df[treated_df['p21_pop'] > 100000]

df_pop_100k.sort_values(by='p21_pop', ascending=False)

In [ ]:
url_df_enseignement_sup = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits-detail_etablissements.csv"

df_enseignement_sup = pd.read_csv(url_df_enseignement_sup, delimiter=';')

df_enseignement_sup.head()

In [ ]:
# 1️⃣ Exclure les lignes avec "Étranger"
df_enseignement_sup = df_enseignement_sup[~df_enseignement_sup['département'].str.contains("Étranger")]

# 2️⃣ Garder uniquement les 2 premiers chiffres
df_enseignement_sup['dep'] = df_enseignement_sup['département'].str[:2]

# 3️⃣ Optionnel : supprimer les espaces et convertir en int
df_enseignement_sup['dep'] = df_enseignement_sup['dep'].str.strip()

df_enseignement_sup.head()

In [ ]:
df_enseignement_sup_treated = df_enseignement_sup.groupby('dep', as_index=False)[[
    'nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE',
    'dont femmes',
    'dont hommes'
]].sum()

df_enseignement_sup_treated.head()

In [ ]:
df_joined = pd.merge(df_pop_100k, df_enseignement_sup_treated, on='dep', how='inner')

df_joined.head()

In [ ]:
df_joined['Densité étudiante'] = df_joined['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'] / df_joined['p21_pop']

df_joined.head()

In [ ]:
# Initialiser le géocodeur
geolocator = Nominatim(user_agent="adam_geocoder_ece_paris")

# Limiteur de requêtes pour éviter les erreurs 429
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_coordinates(ville):
    try:
        if pd.isna(ville):
            return pd.Series([None, None])
        location = geocode(f"{ville}, France")  # ajoute "France" pour plus de précision
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Erreur pour la ville '{ville}': {e}")
        return pd.Series([None, None])

# Appliquer à la colonne des villes
df_joined[["latitude", "longitude"]] = df_joined["libgeo"].apply(get_coordinates)

In [ ]:
df_top_students = df_joined.sort_values(
    by='nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE',
    ascending=False
).head(20)

df_top_density = df_joined.sort_values(
    by='Densité étudiante',
    ascending=False
).head(20)

In [ ]:
plt.figure(figsize=(10, 6)) # Taille du graphique 
plt.bar(df_top_students['libgeo'], df_top_students['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE']) 
plt.title('Top 20 villes par nombre d’étudiants (population > 100k)') 
plt.xlabel('Villes') 
plt.xticks(rotation=45, ha='right', rotation_mode='anchor') 
plt.ylabel('Nombre d’étudiants (en million)') 
plt.grid(False) 

plt.figure(figsize=(10, 6)) # Taille du graphique 
plt.bar(df_top_density['libgeo'], df_top_density['Densité étudiante']) 
plt.title('Histogramme avec courbe de densité pour la colonne X') 
plt.xlabel('Villes') 
plt.xticks(rotation=45, ha='right', rotation_mode='anchor') 
plt.ylabel('Densité Etudiante') 
plt.grid(False) 
plt.show()

In [ ]:
# Carte interactive avec Plotly
fig = px.scatter_map(
    df_joined,
    lat="latitude",
    lon="longitude",
    hover_name="libgeo",
    hover_data=["nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE", "Densité étudiante"],
    size="Densité étudiante",
    zoom=5,
    height=600
)

# Style cartographique (nécessite une Mapbox style)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title="Répartition géographique des étudiants en France")

fig.show()

### Évaluer le taux de vacance locative en France, et les zones propices à un taux élevé ###

In [ ]:
BDD = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main"

df_vac = pd.read_excel(f"{BDD}/insee_rp_hist_1968.xlsx", header=1)
df_pop = pd.read_excel(f"{BDD}/POPULATION_MUNICIPALE_COMMUNES_FRANCE_lucien.xlsx")
df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire

In [ ]:
df_communes_coords = df_communes[[
    "code_insee",       # code INSEE pour faire le merge
    "code_postal",      # code postal
    "latitude_centre",  # latitude du centre de la commune
    "longitude_centre"  # longitude du centre de la commune
]]

In [ ]:
df_vac.columns = ["code_commune", "nom_commune", "annee", "part_log_vacant", "col5", "col6", "col7", "col8"]
df_vac = df_vac[["code_commune", "nom_commune", "annee", "part_log_vacant"]]

df_pop.columns = ["objectid", "reg", "dep", "cv", "codgeo",	"libgeo", "p21_pop"]

df_vac = df_vac.dropna(subset=["part_log_vacant"])
df_vac["part_log_vacant"] = pd.to_numeric(df_vac["part_log_vacant"], errors="coerce")
df_vac = df_vac.dropna(subset=["part_log_vacant"])

df_pop["codgeo"] = df_pop["codgeo"].astype(str)
df_vac["code_commune"] = df_vac["code_commune"].astype(str)

Dataframe des villes de plus de 100 000 habitants classées dans l'ordre croissant par rapport au taux de logement vacant.

In [ ]:
df_vac = df_vac.sort_values(by="part_log_vacant", ascending=True)

df_pop_100k = df_pop[df_pop["p21_pop"] > 100000]

df_vac_100k = df_vac.merge(
    df_pop_100k[["codgeo", "p21_pop"]],
    left_on="code_commune",
    right_on="codgeo",
    how="inner"
)

df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)
df_vac_100k["code_commune"] = df_vac_100k["code_commune"].astype(str)

df_vac_100k = df_vac_100k[["code_commune", "nom_commune", "annee", "part_log_vacant", "p21_pop"]]

In [ ]:
## AFFICHAGE SUR LA MAP


# --- Garder uniquement l'année la plus récente pour chaque commune ---
df_vac_100k_recent = df_vac_100k.sort_values("annee", ascending=False) \
                                 .groupby("code_commune", as_index=False) \
                                 .first()  # prend la ligne la plus récente pour chaque commune

# --- Merge avec les coordonnées ---
df_vac_map = df_vac_100k_recent.merge(
    df_communes_coords,
    left_on="code_commune",
    right_on="code_insee",
    how="left"
).dropna(subset=["latitude_centre", "longitude_centre"])

# --- Affichage carte Plotly ---
import plotly.express as px

fig = px.scatter_mapbox(
    df_vac_map,
    lat="latitude_centre",
    lon="longitude_centre",
    size="part_log_vacant",
    color="part_log_vacant",
    hover_name="nom_commune",
    hover_data={
        "p21_pop": True,
        "part_log_vacant": True,
        "code_postal": True,
        "annee": True
    },
    zoom=5,
    height=500,
    color_continuous_scale="OrRd"
)
# --- Top 10 communes avec le plus petit taux de logements vacants ---
top10_low_vac = df_vac_map.sort_values("part_log_vacant", ascending=True).head(20)

# --- Histogramme ---
fig_hist = px.bar(
    top10_low_vac,
    x="nom_commune",
    y="part_log_vacant",
    text="part_log_vacant",
    hover_data={"p21_pop": True, "code_postal": True, "annee": True},
    labels={"part_log_vacant": "Taux logements vacants", "nom_commune": "Commune"},
    title="Top 10 communes avec le plus petit taux de logements vacants",
    color="part_log_vacant",
    color_continuous_scale="Blues",
    height=600
)

fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig_hist.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_hist.show()

PART 2 : LOYER RENNES VS BDX

PART 3 : LES QUARTIERS